In [1]:
import os
import re
import numpy as np
import tensorflow as tf

from random import shuffle
from glob import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
from itertools import chain
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# Define valid character set
complete_kor_range = range(0xac00, 0xd7a4)
complete_eng_range = range(0x0041, 0x007b)
#punctuations = '~!@#$%^&*()-_=+{}[];:\'"<>,./?'
numbers = '0123456789'

idx_to_chr = []
chr_to_idx = {}
# The first few indices are reserved
idx_to_chr.append('<PAD>')
idx_to_chr.append('<START>')
idx_to_chr.append('<UNK>') # unknown

# A list mapping indices to words
idx_to_chr.extend(numbers)
#idx_to_chr.extend(punctuations)
for i in chain(complete_kor_range, complete_eng_range):
    idx_to_chr.append(chr(i))

# A dictionary mapping charactes to integer indices
for v, k in enumerate(idx_to_chr):
    chr_to_idx[k] = v

In [3]:
def encode_string(s):
    encoded = []
    for c in s:
        try:
            idx = chr_to_idx[c]
        except:
            idx = chr_to_idx['<UNK>']
        encoded.append(idx)
    return encoded


def get_label(s, idx=0):
    label = []
    while True:
        try:
            next_ch = s[idx + 1]
        except:
            # End of sentence
            label.append(0)
            break
        if s[idx] == ' ':
            label.append(1)
            idx += 2
        else:
            label.append(0)
            idx += 1
    return label

# Load model

In [4]:
model = models.load_model('models/02-0.0223-0.9917.hdf5')

W0704 04:45:05.212321 140487151609664 deprecation.py:323] From /home/dustin/anaconda3/envs/tf2/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Make prediction

In [5]:
test = '''영국의철학자인화이트헤드는”서양의 2000년 철학은 모두 플라톤의 각주에 불과하다”라고 말했으며,
          시인 에머슨은 “철학은 플라톤이고, 플라톤은 철학”이라 평하였는데, 플라톤은 소크라테스의 수제자이다. 
          플라톤이 20대인 시절, 스승 소크라테스가 민주주의에 의해 끝내 사형당하는 것을 보고 크게 분개했으며, 
          이는 그의 귀족주의 “철인 정치” 지지의 큰 계기가 되었다. 알렉산더 대왕은 소크라테스의 증손제자로, 
          플라톤의 제자인 아리스토텔레스의 제자'''

In [6]:
max_text_len = 200
regex = re.compile(r'\s+')
test = test.strip().lower()
test = re.sub(regex, '', test)  # remove all whitespaces
print(f'입력: 
{test}')
encoded = encode_string(test)
X_test = pad_sequences([encoded], maxlen=max_text_len, padding='post', truncating='post')

영국의철학자인화이트헤드는”서양의2000년철학은모두플라톤의각주에불과하다”라고말했으며,시인에머슨은“철학은플라톤이고,플라톤은철학”이라평하였는데,플라톤은소크라테스의수제자이다.플라톤이20대인시절,스승소크라테스가민주주의에의해끝내사형당하는것을보고크게분개했으며,이는그의귀족주의“철인정치”지지의큰계기가되었다.알렉산더대왕은소크라테스의증손제자로,플라톤의제자인아리스토텔레스의제자


In [7]:
data = pad_sequences([encoded], maxlen=max_text_len, padding='post', truncating='post')
data

array([[ 6670,   378,  7013,  8365, 10598,  7069,  7045, 10849,  7041,
         9925, 10737,  2281,  1697,     2,  5417,  6558,  7013,     5,
            3,     3,     3,  1361,  8365, 10598,  6989,  3765,  2141,
        10521,  2953,  9649,  7013,    14,  7433,  6621,  4501,   265,
        10597,  1777,     2,  2953,   237,  3549, 10645,  6985,  3709,
            2,  5865,  7045,  6621,  3653,  5813,  6989,     2,  8365,
        10598,  6989, 10521,  2953,  9649,  7041,   237,     2, 10521,
         2953,  9649,  6989,  8365, 10598,     2,  7041,  2953, 10198,
        10597,  6669,  1697,  1917,     2, 10521,  2953,  9649,  6989,
         5529,  9337,  2953,  9561,  5809,  7013,  5669,  7209,  7069,
         7041,  1777,     2, 10521,  2953,  9649,  7041,     5,     3,
         1805,  7045,  5865,  7189,     2,  5809,  5830,  5529,  9337,
         2953,  9561,  5809,    13,  4105,  7433,  7433,  7013,  6621,
         7013, 10625,  1130,  1217,  5305, 10786,  1798, 10597,  1697,
      

In [8]:
data.shape

(1, 200)

In [9]:
p = model.predict(data)

In [10]:
print(f'
출력:')
length = min(len(test), 200)
for i, c in enumerate(test[:length]):
    if p[0][i] >= 0.5:
        print(' ', end='')
    print(c, end='')

영국의 철학자인 화이트 헤드는 ”서양의 2000년 철학은 모두 플라톤의 각주에 불과하다”라고 말했으며, 시인에 머슨은 “철학은 플라톤이고, 플라톤은 철학”이라 평하였는데, 플라톤은 소크라테스의 수제자이다. 플라톤이 20대인 시절, 스승소크라테스가 민주주의에 의해 끝내사 형당하는 것을 보고 크게 분개했으며, 이는 그의 귀족주의 “철인 정치” 지지의 큰 계기가 되었다. 알렉산더 대왕은 소크라테스의 증손제자로 ,플라톤의 제자인 아리스 토텔레스의 제자